In [ ]:
!sudo apt update
!apt-get install openjdk-8-jdk-headless -qq > /dev/null
#Check this site for the latest download link https://www.apache.org/dyn/closer.lua/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!wget -q https://dlcdn.apache.org/spark/spark-3.2.1/spark-3.2.1-bin-hadoop3.2.tgz
!tar xf spark-3.2.1-bin-hadoop3.2.tgz
!pip install -q findspark
!pip install pyspark
!pip install py4j

import os
import sys
# os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
# os.environ["SPARK_HOME"] = "/content/spark-3.2.1-bin-hadoop3.2"


import findspark
findspark.init()
findspark.find()

import pyspark

from pyspark.sql import DataFrame, SparkSession
from typing import List
import pyspark.sql.types as T
import pyspark.sql.functions as F

spark= SparkSession \
       .builder \
       .appName("Our First Spark Example") \
       .getOrCreate()

spark

Hit:1 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease
Hit:2 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease
Hit:3 http://archive.ubuntu.com/ubuntu jammy InRelease
Hit:4 http://archive.ubuntu.com/ubuntu jammy-updates InRelease
Hit:5 http://archive.ubuntu.com/ubuntu jammy-backports InRelease
Hit:6 https://r2u.stat.illinois.edu/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Hit:8 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Hit:9 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubuntu jammy InRelease
Hit:10 https://ppa.launchpadcontent.net/ubuntugis/ppa/ubuntu jammy InRelease
Fetched 129 kB in 2s (61.4 kB/s)
Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
95 packages can be upgraded. Run 'apt list --upgradable' to see them.
W: Skipping acquire of configured file 'main/source/Sources' as repository 'http

# New Section

In [ ]:
!wget https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz

--2025-05-25 18:50:49--  https://static.openfoodfacts.org/data/en.openfoodfacts.org.products.csv.gz
Resolving static.openfoodfacts.org (static.openfoodfacts.org)... 213.36.253.214
Connecting to static.openfoodfacts.org (static.openfoodfacts.org)|213.36.253.214|:443... connected.
HTTP request sent, awaiting response... 302 Moved Temporarily
Location: https://openfoodfacts-ds.s3.eu-west-3.amazonaws.com/en.openfoodfacts.org.products.csv.gz [following]
--2025-05-25 18:50:50--  https://openfoodfacts-ds.s3.eu-west-3.amazonaws.com/en.openfoodfacts.org.products.csv.gz
Resolving openfoodfacts-ds.s3.eu-west-3.amazonaws.com (openfoodfacts-ds.s3.eu-west-3.amazonaws.com)... 52.95.155.60, 3.5.225.173
Connecting to openfoodfacts-ds.s3.eu-west-3.amazonaws.com (openfoodfacts-ds.s3.eu-west-3.amazonaws.com)|52.95.155.60|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 1100200310 (1.0G) [application/gzip]
Saving to: ‘en.openfoodfacts.org.products.csv.gz’

en.openfoodfacts.or 100%[

In [ ]:
!gunzip /content/en.openfoodfacts.org.products.csv.gz

gzip: /content/en.openfoodfacts.org.products.csv already exists; do you wish to overwrite (y or n)? ^C


In [ ]:
df = spark.read.csv("en.openfoodfacts.org.products.csv", header=True, inferSchema=True, sep='\t')
df.printSchema()

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- brands_en: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- catego

In [ ]:
from functools import reduce
from pyspark.sql.functions import col, lower

# Target countries
target_countries = ["morocco", "egypt", "spain", "france","united states"]

# Normalize country tags
df = df.withColumn("countries_tags", lower(col("countries_tags")))

# Build OR condition
country_condition = reduce(
    lambda a, b: a | b,
    [col("countries_tags").contains(c) for c in target_countries]
)

# Apply the filter
filtered_df = df.filter(country_condition)

In [ ]:
filtered_df.printSchema()

root
 |-- code: double (nullable = true)
 |-- url: string (nullable = true)
 |-- creator: string (nullable = true)
 |-- created_t: integer (nullable = true)
 |-- created_datetime: timestamp (nullable = true)
 |-- last_modified_t: integer (nullable = true)
 |-- last_modified_datetime: timestamp (nullable = true)
 |-- last_modified_by: string (nullable = true)
 |-- last_updated_t: integer (nullable = true)
 |-- last_updated_datetime: timestamp (nullable = true)
 |-- product_name: string (nullable = true)
 |-- abbreviated_product_name: string (nullable = true)
 |-- generic_name: string (nullable = true)
 |-- quantity: string (nullable = true)
 |-- packaging: string (nullable = true)
 |-- packaging_tags: string (nullable = true)
 |-- packaging_en: string (nullable = true)
 |-- packaging_text: string (nullable = true)
 |-- brands: string (nullable = true)
 |-- brands_tags: string (nullable = true)
 |-- brands_en: string (nullable = true)
 |-- categories: string (nullable = true)
 |-- catego

In [ ]:
num_rows = filtered_df.count()
num_cols = len(filtered_df.columns)
print(f"Rows: {num_rows}, Columns: {num_cols}")

Rows: 356635, Columns: 209


In [ ]:
important_cols = [
    "code","product_name","image_url","ingredients_text","categories",
    "brands","labels_tags","main_category","countries_tags","origins",
    "manufacturing_places","nutriscore_score","nutriscore_grade","energy_100g",
    "fat_100g","saturated-fat_100g","carbohydrates_100g","sugars_100g",
    "proteins_100g","salt_100g","packaging","additives_tags"
]

essential_cols = [
    "product_name","image_url","ingredients_text","categories",
    "brands","nutriscore_score","nutriscore_grade","energy_100g",
    "fat_100g","sugars_100g","proteins_100g","salt_100g"
]

In [ ]:
df_selected = filtered_df.select(important_cols)
df_cleaned = df_selected.dropna(subset=essential_cols)

In [ ]:
num_rows = df_cleaned.count()
num_cols = len(df_cleaned.columns)
print(f"Rows: {num_rows}, Columns: {num_cols}")

In [ ]:
df_cleaned.write.csv("cleaned_of.csv", header=True, mode="overwrite")

In [ ]:
import os
import shutil

# Path where Spark saved the file
output_dir = "cleaned_of.csv"
output_file = "cleaned_of_single.csv"

# Find the part file and rename it
for file_name in os.listdir(output_dir):
    if file_name.startswith("part") and file_name.endswith(".csv"):
        shutil.move(os.path.join(output_dir, file_name), output_file)
        break

# Remove the Spark output directory
shutil.rmtree(output_dir)